In [34]:
# DL modules
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras import layers
from tensorflow.keras import Sequential, Model
import keras.backend as K
from tensorflow.keras import initializers
! pip install tensorflow_addons
from tensorflow_addons.layers import SpectralNormalization
from tensorflow.keras.optimizers import Adam

# relevent libraries 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

## matplotlib stylings
plt.rcParams['figure.figsize'] = 12, 8

In [36]:
# generator = tf.keras.models.load_model("../Models/CGAN/finalmodel_weights_epoch_951.h5")
# labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
# random_noise = tf.random.normal(shape=(10, 128))
# class_label = keras.utils.to_categorical([0], 10)
# class_label = tf.cast(class_label, tf.float32)
# class_label = tf.repeat(class_label, repeats=10, axis=0)
# noise_and_label = tf.concat([random_noise, class_label], axis=1)
# fake_images = generator(noise_and_label)
# fake_images = tf.image.convert_image_dtype(fake_images, dtype=tf.float32, saturate=True)
# fig, axs = plt.subplots(1, 10, figsize=(10, 1))
# for j in range(10):
#     axs[j].imshow(fake_images[j])
#     axs[j].set_title(labels[j])
#     axs[j].axis("off")
# plt.show()

In [37]:
batch_size = 128
num_channels = 3
num_classes = 10
image_size = 32
latent_dim = 128

In [38]:
generator_in_channels = latent_dim + num_classes
discriminator_in_channels = num_channels + num_classes
print(generator_in_channels, discriminator_in_channels)

138 13


In [39]:
class ConditionalGAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.gen_loss_tracker = keras.metrics.Mean(name="generator_loss")
        self.disc_loss_tracker = keras.metrics.Mean(name="discriminator_loss")

    @property
    def metrics(self):
        return [self.gen_loss_tracker, self.disc_loss_tracker]

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn
        


    def train_step(self, data):
        # Unpack the data.
        real_images, one_hot_labels = data

        # Add dummy dimensions to the labels so that they can be concatenated with
        # the images. This is for the discriminator.
        image_one_hot_labels = one_hot_labels[:, :, None, None]
        image_one_hot_labels = tf.repeat(
            image_one_hot_labels, repeats=[image_size * image_size]
        )
        image_one_hot_labels = tf.reshape(
            image_one_hot_labels, (-1, image_size, image_size, num_classes)
        )

        # Sample random points in the latent space and concatenate the labels.
        # This is for the generator.
        batch_size = tf.shape(real_images)[0]
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Decode the noise (guided by labels) to fake images.
        generated_images = self.generator(random_vector_labels)

        # Combine them with real images. Note that we are concatenating the labels
        # with these images here.
        fake_image_and_labels = tf.concat([generated_images, image_one_hot_labels], -1)
        real_image_and_labels = tf.concat([real_images, image_one_hot_labels], -1)
        combined_images = tf.concat(
            [fake_image_and_labels, real_image_and_labels], axis=0
        )

        # Assemble labels discriminating real from fake images.
        labels = tf.concat(
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))], axis=0
        )

        # Train the discriminator.
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        # Sample random points in the latent space.
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        random_vector_labels = tf.concat(
            [random_latent_vectors, one_hot_labels], axis=1
        )

        # Assemble labels that say "all real images".
        misleading_labels = tf.zeros((batch_size, 1))

        # Train the generator (note that we should *not* update the weights
        # of the discriminator)!
        with tf.GradientTape() as tape:
            fake_images = self.generator(random_vector_labels)
            fake_image_and_labels = tf.concat([fake_images, image_one_hot_labels], -1)
            predictions = self.discriminator(fake_image_and_labels)
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(zip(grads, self.generator.trainable_weights))

        # Monitor loss.
        self.gen_loss_tracker.update_state(g_loss)
        self.disc_loss_tracker.update_state(d_loss)
        return {
            "g_loss": self.gen_loss_tracker.result(),
            "d_loss": self.disc_loss_tracker.result(),
        }

In [40]:
# Create the discriminator.
discriminator = keras.Sequential(
    [
        keras.layers.InputLayer((32, 32, discriminator_in_channels)),
        layers.LeakyReLU(alpha=0.2),
        SpectralNormalization(
        layers.Conv2D(64, kernel_size=4 , strides=2, padding="same"),
        ),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8, epsilon=1e-5),

        SpectralNormalization(
        layers.Conv2D(128, kernel_size=4 , strides=2, padding="same"),
        ),
        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8, epsilon=1e-5),

        SpectralNormalization(
        layers.Conv2D(128, kernel_size=4 , strides=2, padding="same"),
        ),

        layers.LeakyReLU(alpha=0.2),
        layers.BatchNormalization(momentum=0.8, epsilon=1e-5),
        layers.GlobalMaxPooling2D(),
        layers.Flatten(),
        layers.Dropout(0.3),
        layers.Dense(1, activation='sigmoid'),
    ],
    name="discriminator",
)
discriminator.summary()


Model: "discriminator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 leaky_re_lu (LeakyReLU)     (None, 32, 32, 13)        0         
                                                                 
 spectral_normalization (Spe  (None, 16, 16, 64)       13440     
 ctralNormalization)                                             
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 16, 16, 64)        0         
                                                                 
 batch_normalization_18 (Bat  (None, 16, 16, 64)       256       
 chNormalization)                                                
                                                                 
 spectral_normalization_1 (S  (None, 8, 8, 128)        131328    
 pectralNormalization)                                           
                                                     

In [41]:
generator = keras.Sequential(
    [
        keras.layers.InputLayer((generator_in_channels,)),
        # We want to generate 128 + num_classes coefficients to reshape into a
        # 7x7x(128 + num_classes) map.
        layers.Dense(4 * 4 * generator_in_channels),
        layers.ReLU(),
        layers.Reshape((4, 4, generator_in_channels)),
        layers.Conv2DTranspose(64, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(momentum=0.8, epsilon=1e-5),
        layers.ReLU(),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(momentum=0.8, epsilon=1e-5),
        layers.ReLU(),
        layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding="same"),
        layers.BatchNormalization(momentum=0.8, epsilon=1e-5),
        layers.ReLU(),

        layers.Conv2D(3, (4, 4), padding="same", activation="tanh"),
    ],
    name="generator",
)

In [42]:
# generator.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5))

In [45]:
cond_gan = ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=128
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    g_optimizer=keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=False,label_smoothing=0.1),
)

# con_hist = cond_gan.fit(dataset, epochs=1000, callbacks=[CONDGANMonitor(num_img=100)])

In [47]:
cond_gan.generator.load_weights("../Models/CGAN/finalmodel_weights_epoch_951.h5")
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    random_noise = tf.random.normal(shape=(10, 128))
    class_label = keras.utils.to_categorical([i], 10)
    class_label = tf.cast(class_label, tf.float32)
    class_label = tf.repeat(class_label, repeats=10, axis=0)
    noise_and_label = tf.concat([random_noise, class_label], axis=1)
    fake_images = generator(noise_and_label)
    fake_images = tf.image.convert_image_dtype(fake_images, dtype=tf.float32, saturate=True)
    fig, axs = plt.subplots(1, 10, figsize=(10, 1))
    for j in range(10):
        axs[j].imshow(fake_images[j])
        axs[j].set_title(labels[j])
        axs[j].axis("off")
    plt.savefig("{}_images.png".format(labels[i]))


ValueError: Layer count mismatch when loading weights from file. Model expected 8 layers, found 2 saved layers.

In [ ]:
from keras.models import load_model

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
saved_weights = "./finalmodel_weights_epoch_951.h5"
gan_generator = generator.load_weights(saved_weights)
for i in range(10):
    # creating conditions and noise
    conditions = np.full(100, i)
    conditions = tf.keras.utils.to_categorical(conditions, 10)
    latent_z = np.random.normal(size=(100, 128))

    imgs = gan_generator.predict([latent_z, conditions])

    fig = plt.figure(figsize=(20, 20), tight_layout=True)
    for idx, img in enumerate(imgs):
        ax = fig.add_subplot(10, 10, idx+1)
        ax.imshow((img + 1) / 2)
        ax.axis('off')
    fig.suptitle(class_names[i], y=1, fontsize=28)
    fig.savefig(fname="./images/CGAN/{}.jpeg".format(class_names[i]))

ValueError: Layer count mismatch when loading weights from file. Model expected 8 layers, found 2 saved layers.